In [54]:
import os, sys

# Set absolute path to the root folder of the directory
full_path = os.getcwd()
home_folder = 'CPS_GradRate_Analysis'
root = full_path.split(home_folder)[0] + home_folder + '/'
sys.path.append(root)


import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

from sklearn.linear_model import LinearRegression

from src.preprocessing.preprocessing_schoolid import load_prepped_school_year_profile
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
path_to_csv = '../data/chicago_data_portal_csv_files/Chicago_Public_Schools_-_School_Profile_Information_SY1819.csv'
hs_201819 = load_prepped_school_year_profile(path_to_csv)

In [66]:
from sklearn.model_selection import train_test_split, cross_validate

X = hs_201819.drop('Graduation_Rate_School', axis=1)
y = hs_201819.loc[:,'Graduation_Rate_School']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

Find the first simple model in the fsm.ipynb notebook in the same folder as this notebook. It is a simple linear regression which predicts graduation rates of schools in the 2018-19 school year with Student Count Total.   The FSM's R^2 was .186 average across 3 fold crossvalidation.  Student Count Total had a coefficient of .0145, which can be understood as: for every 100 students, the graduation rate goes increases by 1.45 points.

In [75]:
def cps_cross_validate(exogenous_feature_name_list):
    cv = cross_validate(LinearRegression(), X_train[exogenous_feature_name_list], y_train, cv=3, return_train_score=True)
    print('##########train mean R^2#############')
    print(np.mean(cv['train_score']))
    print('##########teast mean R^2#############')
    print(np.mean(cv['test_score']))
    print('#####################################')
    print(cv)


In [76]:
cps_cross_validate(['Student_Count_Total'])

##########train mean R^2#############
0.1855718004689901
##########teast mean R^2#############
0.1424528826778352
#####################################
{'fit_time': array([0.00278711, 0.01644182, 0.00927997]), 'score_time': array([0.00284791, 0.00279903, 0.00433803]), 'test_score': array([0.21600542, 0.14084973, 0.07050349]), 'train_score': array([0.16810904, 0.18307846, 0.2055279 ])}


# Model Iterations

# Add Percent Low Income

In [52]:
X_train['perc_low_income']

204    0.716825
368    0.963470
651    0.890977
290    0.851562
5      1.000000
         ...   
317    0.803797
500    0.941935
48     0.794355
432    0.955307
483    0.969754
Name: perc_low_income, Length: 105, dtype: float64

In [77]:
cps_cross_validate(['Student_Count_Total', 'perc_low_income'])

##########train mean R^2#############
0.19275238683420123
##########teast mean R^2#############
0.13335856366759827
#####################################
{'fit_time': array([0.00633693, 0.00397992, 0.00354004]), 'score_time': array([0.00341296, 0.00183678, 0.0022862 ]), 'test_score': array([0.22482208, 0.14596739, 0.02928623]), 'train_score': array([0.17230852, 0.18440707, 0.22154157])}


> adding percent low income actually reduces the overal score on the test.

# Add Dress Code

In [78]:
cps_cross_validate(['Student_Count_Total', 'perc_low_income', 'Dress_Code'])

##########train mean R^2#############
0.2856334081801379
##########teast mean R^2#############
0.15379413386672489
#####################################
{'fit_time': array([0.00767994, 0.00825691, 0.00321817]), 'score_time': array([0.00372291, 0.0031991 , 0.00310898]), 'test_score': array([0.37651082, 0.08087026, 0.00400132]), 'train_score': array([0.20766878, 0.32733451, 0.32189694])}


> Adding dress code improves the train score, but there is still a big drop off, with very little predictive power in two of the validation sets.